# Simple preprocessing


Hello People! 

Welcome to the first Notebook. 

Like we described in our presentation. This is notebook is used for the creation of easy features. 

__What is an easy feature?__

You can think of it like a column in one of the data files. At the beginning
we have the three input files (items, info, orders). In the end we will export 2 files: 
- ```dynamic_features.pkl```-> All time dependent data such as date features (e.g day_of_month)
- ```static_features.pkl```-> All static features such as itemID and so on

After this notebook we **only** work with ```dynamic_features.pkl``` and ```static_features.pkl```.

_Why the fuck ```*.pkl```?!_ 

pkl in this case stands for 'pickle' and allows us to very space efficiently safe the data in bytes. When loading the data we don't have the problem with delimiters like we would have with csv. But most important: Later we need to save models and so on with pickle. So we already start here using it to have some consistency in these unconsistent times :D

__How do we handle our test set ?__
As we already tried out in team 1, it is crucial to think about the complexity of test features beforehand. Therefore, we now prepare the test set from right the beginning. 

This means that we now already created rows for the testset. This means the aforementioned files then consist out of **all** the data (test + validation + train). _But why?!_ In the first two notebooks we have function (later more about those) to mutate our data. These mutation then directly can be done for test and train set, using the same functions. This finally leads to correct function. Furthermore, everyone who is adding a new feature now already needs to think about, how it may be applied to the train set (in which we have not all information)

How the creation of testset features works, will be elaborated later. 

__What is the structure of this notebook?__

1. **Setting up Colab**
1. **Notes and Ideas**
1. **Setting up Notebook**
1. **Create Dynamic_Features Dataframe** 
1. **Deviation of Promotions and Creation of Test Set**
1. **Defining Feature Functions**
1. **Applying Features**
1. **Testset Mappings**
1. **Saving data**

__TL;DR__
- Input = info.csv/ items.csv / orders.csv
- Output = dynamic_features.csv / static_features.csv
- Notebook for easy features (new Columns in source data)
- Test set is simulated and created already here.


# 1. Setting up Colab

Here we set up colab. U knooow it :D 

In case u don't use colab, please specify the variable ```use_colab``` to false. In case u don't use it its unavoidable to make sure that noone changed something online while u change it offline. The results of this would be worse than a division by zero.  

In [1]:
use_colab: bool = True

In case u accidentall run the following code twice u will get following weird and confusing error:

```
OSError: [Errno 107] Transport endpoint is not connected
```

In this case just restart the runtime above. 


In [2]:
if use_colab:
  import os
  from google.colab import drive 
  drive.mount("/content/gdrive", force_remount=True)
  # Change directory for nicer imports
  %cd "/content/gdrive/My Drive/Data_Mining_Cup/05 Code/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Data_Mining_Cup/05 Code


# 2. Notes and Ideas

## Ideas for new Features
Please specify assignment in bold and brackets behind. If nobody is assigned put the "To Be Assigned (TBA)" flag. 

- Boolean two days after promotion (Nic)
- how rare are solds **(TBA)**
- Sum of solds 14 days after -> Can be used for the direct prediction of sum of values **(TBA)**


## Notes

Currently no notes :( 

# 3. Setting up Notebook

Here we setup some parameters for the notebook to work in the expected ways.

If we want to **use the final test phase** for the later submission, we need to specify the following parameter **```make_final_submission``` to true.** Otherwise the selfmade test set will be used.

In [3]:
make_final_submission: bool = False

In case we don't want to make a final submission, we now have to specify the test_set. In case you change it, please leave a comment on why and what the new one is. If the end is defined as None we use all the data from start_period (inclusive) till the end of dataset. 

In case make_final_submission is True we do not use those parameters.

In [4]:
# This will set the start of test period to 2 weeks before end of data
test_period_start: str="2018-06-02"
# Use till the end
test_period_end: str="2018-06-15"

In [5]:
# These datapoints are for the final submision
final_submission_start: str="2018-06-30"
final_submission_end: str="2018-07-13"  # shouldn't it be 07-13 if inclusive? 

#### Imports

Here you can add imports u need. Please use alphabetical order to not import new stuff twice.

In [6]:
from datetime import datetime, timedelta

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from typing import List, Callable

%matplotlib inline

### Data Imports

Now we import the source data. In case we use our test set, we delete the test set values to simulate later submission.

In [7]:
# In tables 'infos' and 'items' itemID is used as index
info = pd.read_csv('/content/gdrive/My Drive/Data_Mining_Cup/02 Data/DMC20_Data/infos.csv', delimiter='|', index_col="itemID")
items = pd.read_csv('/content/gdrive/My Drive/Data_Mining_Cup/02 Data/DMC20_Data/items.csv', delimiter='|', index_col="itemID")

In [8]:
orders = pd.read_csv('/content/gdrive/My Drive/Data_Mining_Cup/02 Data/DMC20_Data/orders.csv', delimiter='|', parse_dates=True)
orders['time'] = pd.to_datetime(orders['time'])

# 4. Create DataFrame ```dynamic_features```
Here we create the data frame for dynamic features out of the orders dataframe. Following the orders dataframe can only be used to add something to this dataframe. 
As indexes it has the date as well as the itemID.

In [ ]:
orders['date'] = orders['time'].dt.date
dynamic_feature = orders.groupby(['itemID','date']).aggregate({'order':'sum', 'salesPrice':'mean'})

In [ ]:
test = dynamic_feature.reset_index()

In [ ]:
itemID_list = test['itemID'].unique()

In [ ]:
itemID_list

array([    1,     2,     3, ..., 10460, 10462, 10463])

In [ ]:
# Create a new dataframe to store time-dependent data for items
_begin = min(orders['time']).date()
_end = max(orders['time']).date()
timespan = [p.date().strftime('%Y%m%d') for p in pd.date_range(start=_begin, end=_end)]
idx=  pd.MultiIndex.from_product([items.index, pd.to_datetime(timespan)], names=['itemID', 'date'])

In [ ]:
dynamic_feature = pd.DataFrame(dynamic_feature, index=idx)
dynamic_feature['order'] = dynamic_feature['order'].fillna(0)
dynamic_feature['salesPrice'] = dynamic_feature['salesPrice'].groupby('itemID').apply(lambda group: group.fillna(items.loc[group.name].recommendedRetailPrice))

# using the following forward & backward fill method would cause wrong prices (changed to the above line instead)
# e.g. itemID 58 was never sold thus not appeared in the orders table, then it would get the price from itemID 57 
# dynamic_feature['salesPrice'] = dynamic_feature['salesPrice'].fillna(method="bfill")
# dynamic_feature['salesPrice'] = dynamic_feature['salesPrice'].fillna(method="ffill")

# 5. Deviation of Promotions and Creation of Test Set

Here we derive the promotions using the following method. This needs to be done to be able to have promotions in the test set when we manipulate it. 

#### Promotion Derivation Functions

The following functions can be used for deriving the promotions for test set. However, these function can also be used later to derive promotions for train set.

In [ ]:
def add_promotions_via_log2_new(lam=1, limit=2.5, **kwargs)->pd.DataFrame:
  """
  data = timedep
  Predicts when items were promoted according to number of sold items on a daily
  basis.
  As promotions are marked all days where the follwing holds:
  log2((sold_on_current_day + lambda) / (sold_on_prev_day + lambda)) > limit
  """
  def promotion_function(data: pd.DataFrame, ors: pd.DataFrame=None):
    print("... adding promotions")
    sold_shifted = data['order'].shift(periods=1)
    trend = np.log2((data['order'] + lam) / (sold_shifted + lam))
    data['promoted'] = trend > limit
    print(f"..... There are {sum(data['promoted'])} promotions derived")
    return data
  return promotion_function

In [ ]:
# OLD FUNCTION
def add_promotions_via_log2(data: pd.DataFrame, ors: pd.DataFrame, lam=1, **kwargs)->dict:
  """
  data = timedep
  Predicts when items were promoted according to number of sold items on a daily
  basis.
  As promotions are marked all days where the follwing holds:
  log2((sold_on_current_day + lambda) / (sold_on_prev_day + lambda)) > limit
  """
  print("... adding promotions")
  # Add the timestamp as index
  ors = ors.set_index(pd.DatetimeIndex(ors['time']))

  # Hyperparameter for smoothing
  idx = None

  grouped = ors.groupby(['itemID'])
  for item_id, group in grouped:
    if item_id % 500 == 0:
      print(f"   ... {item_id}/{len(grouped)}")
    sold_each_day = group['order'].resample('D').sum()
    first_day = sold_each_day.index[0]

    if first_day.day == 1 and first_day.month == 1:
      # This is the first day of the simulation, the day before is unknown
      fill_value = np.nan
    else:
      # The item was not bought on the day before
      fill_value = 0
    
    # Use log difference (see https://www.youtube.com/watch?v=_N88aMUjDb8&feature=youtu.be, ~8m15s)
    # Shift for one day
    sold_each_day_shifted = sold_each_day.shift(periods=1, fill_value=fill_value)

    trend = np.log2((sold_each_day + lam) / (sold_each_day_shifted + lam))

    promoted = trend[trend > 2.5]
    promoted_dates = list(map(lambda x: x.to_pydatetime().strftime('%Y%m%d'), promoted.index))

    _idx = pd.MultiIndex.from_product([[item_id], promoted_dates])
    if idx is None:
      idx = _idx
    else:
      idx = idx.union(_idx)
  
  data.loc[idx, 'promoted'] = True
  # data.fillna(value={'promoted': False}, inplace=True)
  data['promoted'].fillna(False, inplace=True)

  return data

In [ ]:
def add_promotions_via_naive_mean_deviation(data, **kwargs):
  data["avg"] = data["order"].mean()
  data["promotion2"] = 0
  data.loc[data["avg"]< data["order"],"promotion2"] = 1
  data.drop(columns=['avg'], inplace = True)
  return data

### Apply Promotion Derivation

The following parameter defines which function is used to derive the promotions:

In [ ]:
function_for_promotion: Callable = add_promotions_via_log2_new(limit=2.5)

---
Now we are adding the promotions to all of the data

In [ ]:
dynamic_feature = function_for_promotion(data=dynamic_feature)

... adding promotions
..... There are 17596 promotions derived


#### Add Testing Columns

Here we are also saving the orders which should be predicted. These are needed to check the performance of trained models.

In [ ]:
if make_final_submission:
  # Set test range
  test_range =  pd.date_range(start=final_submission_start, end=final_submission_end)
  # Create index for test
  idx = pd.MultiIndex.from_product([items.index, pd.to_datetime(test_range)], names=['itemID', 'date'])
  # Create DataFrame for the ending
  dynamic_feature_test = pd.DataFrame(index=idx, columns=dynamic_feature.columns)

  # Add the simulation price
  simulation_price2 = pd.DataFrame(info['simulationPrice'].rename('salesPrice'))
  # Add simulation Price
  dynamic_feature_test = dynamic_feature_test.drop('salesPrice', axis=1).join(pd.DataFrame(simulation_price2), on='itemID')
  dynamic_feature_test['promoted'] = False
  
  # Add promotions
  for idx, row in info.iterrows():
    if not(str(row['promotion'])=="nan"):
      promotions = row['promotion'].split(",")
      for date in promotions:
        dynamic_feature_test.loc[(idx, pd.to_datetime(date)),'promoted']=True
    
else:
  #test_range = pd.date_range(start=test_period_start, end=test_period_end)
  dynamic_feature_test = dynamic_feature[dynamic_feature.index.get_level_values(1) >= pd.to_datetime(test_period_start)]
  dynamic_feature = dynamic_feature[dynamic_feature.index.get_level_values(1) < pd.to_datetime(test_period_start)]
  simulation_price = pd.DataFrame(dynamic_feature_test['salesPrice'].groupby('itemID').mean())
  dynamic_feature_test = dynamic_feature_test.drop('salesPrice', axis=1).join(pd.DataFrame(simulation_price), on='itemID')
  
  # Save the testing orders 
  dynamic_feature_test.to_pickle("./data/orders_for_test.pkl")
  

  # Delete Order in test set
  dynamic_feature_test['order'] = np.NaN

dynamic_feature = dynamic_feature.append(dynamic_feature_test)

del dynamic_feature_test

In [ ]:
dynamic_feature

order  salesPrice  promoted
itemID date                                   
1      2018-01-01    0.0        3.11     False
       2018-01-02    0.0        3.11     False
       2018-01-03    0.0        3.11     False
       2018-01-04    0.0        3.11     False
       2018-01-05    0.0        3.11     False
...                  ...         ...       ...
10463  2018-06-25    NaN      282.16     False
       2018-06-26    NaN      282.16     False
       2018-06-27    NaN      282.16     False
       2018-06-28    NaN      282.16     False
       2018-06-29    NaN      282.16     False

[1883340 rows x 3 columns]

In the ```orders``` dataframe, we are not able to get information for prediction, because in later submission we don't neither. Therefore we need to delete rows, which define rows in the later notebook. 

In [ ]:
if not make_final_submission:
    orders = orders[orders['time'] < test_period_start]

# 6. Defining Feature Functions

In this area new features can be defined. A feature function takes the data as argument and returns the mutated dataframe. Following attributes can be used additionally:

- ```ors```: The orders dataframe
- ```its```: The items dataframe

__Why do we need functions?__

This has three main reasons:
- We can easily add and delete functions
- We have an overview on which features are used currently
- The features we create here can be used in other projects as well 

__What if my function needs additional parameters?__

Then create a function which returns a the final function. The following function returns a function which changes the sales price to the default value + 10.

```
def test_function(default_value=1):

  def final_function(data: pd.DataFrame):
    data['salesPrice'] = default_value + 10

    return data
  return final_function

```
Later this function can be triggered using following:

```
test_function(default_value=10)(data)
```
This structure is important for later and enables adding the same feature with only different parameter.

__How do I import features which are created by other regressors models or whatever?__

Then please save the feature column in a seperate file. Then import the column in a function and then add it to the data like in the other feature function. In case that u simply import a file, specify how this feature is created, and please give a link or description on the location of the source code of the creation of this file. 
Furthermore, please only import files, which are accessible by all of the team. So that in the end every team member is able to add the feature and run all of the pipeline. 

#### Has Rating Ordinal Feature

Adds True if there is a rating.

In [ ]:
def add_has_rating_feature(data: dict, **kwargs)->dict:
  """
  Ordinal feature, which returns if a method has a rating
  """
  print("... applying has rating")
  data['has_rating'] = data['customerRating']> 0
  return data

#### Summed orders per item

Feature which describes, how much was brought 2 weeks after this day. This feature can then be used for predicting the whole simulation period. 

In [ ]:
# dynamic_feature['order'].iloc[i]

In [ ]:
#####
# UNDER CONSTRUCTION
# -- probably won't use 
###

# def summed_order_in_item(time_steps=14):
  
#   def feature_function(data,**kwargs):
#     test = data['order'].rolling(time_steps).sum()
#     return test
#   return feature_function


# this function is not applied at the very end, do we still want to use it?
# the number could be wrong as it's not grouped by each item 
# also need to fill NaN values at the very beginning for the rolling time_steps 

In [ ]:
#summed_order_in_item()(data=dynamic_feature)

#### Add Divided Time Feature

Adds time information:
- Timestamp
- Time
- Date
- Hour
- Day_of_year
- Day_of_month
- Month
- Week_nr
- Day_of_Week

In [ ]:
def add_divided_time_feature(data: dict, **kwargs)->dict:
  """
  data = dynamic_feature

  Divides the timestamp of the data into different features like the weekday etc.
  """
  print("... adding time features")

  date = data.index.get_level_values(1)

  data['day_of_year'] = date.dayofyear
  data['day_of_month'] = date.day
  data['month'] = date.month
  data['week_nr'] = date.week # updated here, week_nr should be int instead of str
  data['day_of_week'] = date.dayofweek # int(Monday:0 Sunday:6)
  return data

#### Aggregate Sold Feature

Adds daily_sold, weekly_sold and monthly_sold items

In [ ]:
def add_aggregated_sold_feature(data:dict, **kwargs)->dict: 
  print('... applying sold features')
  # add sold per day/week/month column to orders
  data = data.assign(daily_sold = data.groupby(['date', 'itemID']).order.transform('sum'))
  data = data.assign(weekly_sold = data.groupby(['week_nr', 'itemID']).order.transform('sum'))
  data = data.assign(monthly_sold = data.groupby(['month', 'itemID']).order.transform('sum'))
  return data

####Add_avg_salesPrice
Adds the average salesprice over the whole training and validation_period

TODO: Check if not also test period is within -> If this is the case, its not tragic cause we use the sim_price for sales price in testing

In [ ]:
def add_avg_salesPrice(data, ors, **kwargs):
  """
  data = its
  Calculates the avg sales price over the whole training and validation period
  """
  print("... adding avg sales price")

  # Calculate average over all orders (non-unique)
  avg_salesPrice_nonunique = ors[['salesPrice', 'itemID']].groupby('itemID').mean().rename(columns={'salesPrice':'avg_salesPrice_nonunique'})
  data = data.join(avg_salesPrice_nonunique, on="itemID")
  data['avg_salesPrice_nonunique'].fillna(data['recommendedRetailPrice'], inplace=True)
  return data

#### Add Avg Daily Sales Price

Adds the daily sales price for each item

In [ ]:
def add_avg_daily_salesPrice(data, ors, its, **kwargs):
  """
  data = timedep
  Calculates the average of (unique) prices for every day.
  """
  print("... adding daily sales price")

  grouped = ors.groupby(['itemID', 'date'])['salesPrice']

  # Add average of unique prices
  daily_price_item = grouped.unique().apply(lambda x: np.mean(x))
  data.loc[daily_price_item.index, 'avg_daily_salesPrice'] = daily_price_item

  # Add average of all prices (non-unique)
  daily_price_item_nonunique = grouped.mean()
  data.loc[daily_price_item_nonunique.index, 'avg_daily_salesPrice_nonunique'] = daily_price_item_nonunique

  # If value is missing, first check if there is average of the whole period. -> would not be null 
  data['avg_daily_salesPrice'] = data['avg_daily_salesPrice'].groupby('itemID').apply(lambda group: group.fillna(its.loc[group.name].avg_salesPrice_nonunique))
  data['avg_daily_salesPrice_nonunique'] = data['avg_daily_salesPrice_nonunique'].groupby('itemID').apply(lambda group: group.fillna(its.loc[group.name].avg_salesPrice_nonunique))

  return data

#### Add Avg of daily salesPrice
Calculate average over daily averages.

In [ ]:
def add_avg_of_daily_salesPrice(data, td, **kwargs):
  """
  data = items
  Calculate average over daily averages (nonunique)
  """
  avg = td.groupby('itemID')['avg_daily_salesPrice_nonunique'].mean()
  data.loc[avg.index, 'avg_salesPrice'] = avg
  return data

#### Add Avg Daily Sold
  data = items
  
  Calculate avg daily sold and standard deviation.


In [ ]:
def add_avg_daily_sold(data, ors, **kwargs):
  """
  data = items
  Calculate avg daily sold and standard deviation.

  """
  print("... adding avg daily sold")
  nr_days_training = (ors['time'].max() - ors['time'].min()).days + 1
  daily_sold = ors.groupby(['itemID', 'date']).order.sum()
  grouped = daily_sold.groupby('itemID')

  # Avg over all training period
  avg_sold_item = (grouped.sum() / nr_days_training)
  data.loc[avg_sold_item.index, 'avg_sold_day'] = avg_sold_item

  # Avg over days where total number of orders > 0
  avg_sold_item_only_nonzero = grouped.mean()
  data.loc[avg_sold_item_only_nonzero.index, 'avg_sold_day_only_nonzero'] = avg_sold_item_only_nonzero
  
  # Std over days where total number of orders > 0
  std_sold_item_only_nonzero = grouped.std()
  data.loc[std_sold_item_only_nonzero.index, 'std_sold_day_only_nonzero'] = std_sold_item_only_nonzero

  # Fill missing values
  data['avg_sold_day'].fillna(0, inplace=True)
  data['avg_sold_day_only_nonzero'].fillna(0, inplace=True)
  data['std_sold_day_only_nonzero'].fillna(0, inplace=True)
  return data

#### Add_SalesPrice_Over_Price
TODO: Please add description and assertions for wrong order (Just test if add_avg_daily_salesPrice_nonunique & add_avg_salesPrice are present, if not tell that the other functions need to be executed before)

TODO: The function which should be executed, are not existing!

@ Markus, the function for creating avg_daily_salesPrice_nonunique indeed does exist (check out function add_avg_daily_salesPrice()), also avg_salesPrice should also be created within the function add_avg_of_daily_salesPrice(). avg_salesPrice should be the average of avg_daily_salesPrice_nonunique. I hope it makes sense (Min). 

In [ ]:
# Execute after: def add_avg_of_daily_salesPrice & add_avg_of_daily_salesPrice

# I added "salesPrice_over_avg_salesPrice" and "salesPrice_over_avg_salesPrice_nonunique" (Min)

def add_salesPrice_over_price(data, its, **kwargs):
  """
  data = timedep or dynamic features
  calculate relationships between price of that day and [recommendedRetailPrice, avg_salesPrice, and avg_salesPrice_nonunique]
  """
  print("... applying add_salesPrice_over_price")

  df_sales = data.join(its, on = 'itemID', how = 'left')
  data['salesPrice_over_recommended'] = df_sales['avg_daily_salesPrice_nonunique'] / df_sales['recommendedRetailPrice']
  data['salesPrice_over_avg_salesPrice'] = df_sales['avg_daily_salesPrice_nonunique'] / df_sales['avg_salesPrice']
  data['salesPrice_over_avg_salesPrice_nonunique'] = df_sales['avg_daily_salesPrice_nonunique'] / df_sales['avg_salesPrice_nonunique']
  
  data['salesPrice_over_recommended'].fillna(0, inplace=True)
  data['salesPrice_over_avg_salesPrice'].fillna(0, inplace=True)
  data['salesPrice_over_avg_salesPrice_nonunique'].fillna(0, inplace = True)

  return data

#### Add German holidays

Method to add German Holidays

In [ ]:
def add_german_holidays(data, **kwargs):
  
  import holidays
  print("... applying add_german_holidays")
  de_holidays = holidays.DE(years= 2018) #Holidays Library
  a = pd.DataFrame(de_holidays.keys())

  # Creating bool: if date is holiday
  data['de_holidays'] = data.index.get_level_values(1).isin(a)

  return data

#### Add Has Promotion on Day

Adds a column to orders that tells whether the item in this row has been 
  promoted on that day.

In [ ]:
def add_has_promotion_on_day(data: dict, its: dict, **kwargs):
  """
  Adds a column to orders that tells whether the item in this row has been 
  promoted on that day.
  """
  print("... applying add_has_promotion_on_day")
  data = data.assign(has_promotions_this_day=False)
  grouped = data.groupby(['itemID', 'date'])
  for (item_id, date), group in grouped:
    date_correct_format = group.iloc[0]['time'].to_pydatetime().strftime('%Y-%m-%d')
    if date_correct_format in its.loc[item_id].promotion_derived.split(','):
      data.loc[group.index, ['has_promotions_this_day']] = True
  return data

## Num Promotions

In [ ]:
def num_promotions(data, td, its, **kwargs):
  print("... adding num promotions")
  num_promotions = td[td['promoted']==True].groupby('itemID')['promoted'].count()
  num_promotions.name = 'num_promotions'
  data = data.join(num_promotions, rsuffix="_num_promotions")
  data['num_promotions'].fillna(value=0,inplace=True)
  return data

## Promotion in Data

In [ ]:
# add after num_promotions
def promotion_in_data(data, td, its, **kwargs):
  print("... adding promotion in data")
  print(td)
  data['promotion_in_data'] = data['num_promotions'].astype('bool')
  return data

## Avg Sold Promoted

In [ ]:
def avg_sold_promoted(data, td, its, **kwargs):
  print("... adding avg_sold_promoted")
  promoted = td[td['promoted']==True].groupby('itemID')['order'].mean()
  promoted.name = 'avg_sold_promoted'
  data = data.join(promoted, rsuffix="_promoted")
  data['avg_sold_promoted'].fillna(value=data['avg_sold_promoted'].mean(),inplace=True)
  return data

## Avg Sold Not Promoted

In [ ]:
def avg_sold_not_promoted(data, td, its, **kwargs):
  print("... adding avg_sold_not_promoted")
  not_promoted = td[td['promoted']==False].groupby('itemID')['order'].mean()
  not_promoted.name = 'avg_sold_not_promoted'
  data = data.join(not_promoted)
  data['avg_sold_not_promoted'].fillna(value=data['avg_sold_not_promoted'].mean(),inplace=True)
  return data

## Last promotions

In [ ]:
def last_promotion(item, avg_sales_of_all_items):
  itemID = item.index.values[0][0]
  if itemID % 1000 == 0:
    print(itemID)

  first_day_of_data = item.index.get_level_values(1).to_list()[0]
  last_day_of_data = item.index.get_level_values(1).to_list()[-1]

  def get_avg_order_on_promo_day(promos):
    idx = pd.MultiIndex.from_product([[itemID], promos])
    orders_all = item.loc[idx]
    orders = orders_all[orders_all.order.notna()].order
    if len(orders) > 0:
      avg = orders.mean()
    else:
      avg = avg_sales_of_all_items
    return round(avg)

  def helper(start_date, end_date, default_sales=None):
    # Set values to entries between start_date and end_date
    days_between = (end_date - start_date).days
    first_day_to_set = start_date + timedelta(days=1)  # First day after last promotion

    if first_day_to_set < first_day_of_data:
      value_for_january_1st = (first_day_of_data - first_day_to_set).days + 1
      first_day_to_set = first_day_of_data
      days_since_last_promo = np.arange(value_for_january_1st, days_between+1)
      last_promo_sales = default_sales  # Avg of promotions for this item
    else:
      days_since_last_promo = np.arange(1, days_between+1)  # A list of increasing numbers, starting with 1
      last_promo_sales = item.loc[(itemID, start_date)].order  # Sales on the last promoted day

    item.loc[(itemID, first_day_to_set):(itemID, end_date), "days_since_last_promotion"] = days_since_last_promo    
    item.loc[(itemID, first_day_to_set):(itemID, end_date), "last_promoted_sales"] = last_promo_sales
    return item

  # Get list of dates when the item was promoted
  promotions = item.index[item.promoted].get_level_values(1).to_list()

  default_sales = get_avg_order_on_promo_day(promotions)

  last_promo = datetime(year=2017, month=12, day=1)  # First imaginary promotion

  if len(promotions) == 0:
    item = helper(last_promo, last_day_of_data, default_sales)
    return item

  for promo_day in promotions:
    # Loop through all promotions and set all intermediate values at once
    item = helper(last_promo, promo_day, default_sales)
    last_promo = promo_day

  # Add values for the days after the last promotion
  
  if (last_day_of_data - last_promo).days > 0:
    helper(last_promo, last_day_of_data)

  return item

In [ ]:
def last_promotions(data, **kwargs):
  print("... adding last_promotions")
  data["days_since_last_promotion"] = np.nan
  data["last_promoted_sales"] = np.nan
  avg_sales_of_all_items = data[data['promoted']==True].groupby('itemID')['order'].mean().mean()
  data = data.groupby('itemID').apply(last_promotion, avg_sales_of_all_items = avg_sales_of_all_items)
  return data

## Creating Price Clusters

Creation of price buckets

In [ ]:
# for recommendedRetailprice in items
def add_price_bins(n_bins: int, field: str):
  def bin_fct(data, **kwargs):
    print(f"... applying price bins on {field} with {n_bins} bins")
    data[f'{field}_bucket'] = pd.qcut(data[field], n_bins, labels=range(0, n_bins))
    return data
  return bin_fct

## Creating Price/Order Clusters

In [ ]:
# Using sum of all orders per item
def add_price_sumOrder_cluster(data, ors, **kwargs):
  # implement number of orders from order file 
  sold_each_item = ors[['itemID', 'order']].groupby(['itemID']).sum()
  data = data.reset_index()
  data = data.join(sold_each_item, on="itemID")
  # Clustering
  n_bins = 3
  data['priceCluster'] = pd.qcut(data.recommendedRetailPrice, n_bins, labels=range(0, n_bins))
  data['orderCluster'] = pd.qcut(data.order, n_bins, labels=range(0, n_bins))
  # drop all rows with NaN in orderCluster  
  data = data[data['orderCluster'].notna()]
  
  # Combining price and order Cluster
  def combi_pr_or(row):
      global val
      if row['priceCluster'] == 0 and row['orderCluster'] == 0:
          val = 1 #cheap and sold rarely
      elif row['priceCluster'] == 0 and row['orderCluster'] == 1:
          val = 2 #cheap and sold middle
      elif row['priceCluster'] == 0 and row['orderCluster'] == 2:
          val = 3 #cheap and sold often
      elif row['priceCluster'] == 1 and row['orderCluster'] == 0:
          val = 4 #middle price and sold rarely
      elif row['priceCluster'] == 1 and row['orderCluster'] == 1:
          val = 5 #middle price and sold middle
      elif row['priceCluster'] == 1 and row['orderCluster'] == 2:
          val = 6 #middle price and sold often
      elif row['priceCluster'] == 2 and row['orderCluster'] == 0:
          val = 7 #expensive and sold rarely
      elif row['priceCluster'] == 2 and row['orderCluster'] == 1:
          val = 8 #expensive and sold middle
      elif row['priceCluster'] == 2 and row['orderCluster'] == 2:
          val = 9 #expensive and sold often
      return val

  # Combine both Cluster with combi_pr_or function
  data['price_sumSales_cluster'] = data.apply(combi_pr_or, axis=1)
  data = data.drop(['priceCluster', 'orderCluster', 'order'], axis=1)
  data = data.set_index("itemID")

  return data

In [ ]:
# Using max of Order per item
def add_price_maxOrder_cluster(data, ors, **kwargs):
  # find max order for every itemID in save as order in max_orders
  max_orders = ors.loc[ors.reset_index().groupby(['itemID'])['order'].idxmax()]
  data = data.reset_index()
  data = pd.merge(data, max_orders[['order', 'itemID']], on="itemID", how='left')
  # Clustering
  n_bins = 3
  data['priceCluster'] = pd.qcut(data.recommendedRetailPrice, n_bins, labels=range(0, n_bins))
  data['orderCluster'] = pd.qcut(data.order, n_bins+1, labels=range(0, n_bins), duplicates='drop')
  
  # drop all rows with NaN in orderCluster  
  data = data[data['orderCluster'].notna()]

  # Combining price and order Cluster
  def combi_pr_or(row):
      global val
      if row['priceCluster'] == 0 and row['orderCluster'] == 0:
          val = 1 #cheap and sold rarely
      elif row['priceCluster'] == 0 and row['orderCluster'] == 1:
          val = 2 #cheap and sold middle
      elif row['priceCluster'] == 0 and row['orderCluster'] == 2:
          val = 3 #cheap and sold often
      elif row['priceCluster'] == 1 and row['orderCluster'] == 0:
          val = 4 #middle price and sold rarely
      elif row['priceCluster'] == 1 and row['orderCluster'] == 1:
          val = 5 #middle price and sold middle
      elif row['priceCluster'] == 1 and row['orderCluster'] == 2:
          val = 6 #middle price and sold often
      elif row['priceCluster'] == 2 and row['orderCluster'] == 0:
          val = 7 #expensive and sold rarely
      elif row['priceCluster'] == 2 and row['orderCluster'] == 1:
          val = 8 #expensive and sold middle
      elif row['priceCluster'] == 2 and row['orderCluster'] == 2:
          val = 9 #expensive and sold often
      return val
    
  # Combine both Cluster with combi_pr_or function
  data['price_maxSales_cluster'] = data.apply(combi_pr_or, axis=1)
  data = data.drop(['priceCluster', 'orderCluster', 'order'], axis=1)
  data = data.set_index("itemID")

  return data

#### Clusters for items that were frequently bought together

In [ ]:
def add_bought_together_clusters(data, **kwargs):
  """
  data = items

  Items that were "often" bought together are in the same cluster.
  What "often" means can be changed in the nb linked below. The clusters that 
  are currently used are based on parameters (10, 40): 
  - two items must be bought together at least 40 times.
  - at least 10 percentiles of times when one item was bought, the other item 
    was also bought.
  (the described approach is similar to apriori algorithm)

  I made the relation transitive: if A together with B and B together with C,
  then A together with C, which means that all three (A, B, C) are in the same 
  cluster.

  Cluster value -1 means no cluster (the item was rarely or never bought 
  together with something else).

  Link to the nb where this is derived: https://colab.research.google.com/drive/1L0LEBlxD-X9wvyH28XQNvQTmQDq68sYl
  """
  print("... adding bought_together_cluster")
  clusters = pd.read_pickle('./data/bought_together.pk')
  data['bought_together_cluster'] = clusters['together_10_40']
  return data

In [ ]:
def add_two_days_after_promo(data, **kwargs):
  """
  add boolean variable for if date is one or two days after promo, as after an auction items stay promoted for 44 hours
  """
  data.reset_index(inplace=True)
  data.set_index('itemID', inplace = True)
  data["promotion_lag1"] = data.groupby(data.index).promoted.shift(1)
  data["promotion_lag2"] = data.groupby(data.index).promoted.shift(2)
  data["promotion_lag1"].fillna(False, inplace = True)
  data["promotion_lag2"].fillna(False, inplace = True)
  data["two_days_after_promo"] = data["promotion_lag1"]
  data.loc[data["promotion_lag2"]==True,"two_days_after_promo"] = True
  data.reset_index(inplace=True)
  data.set_index(["itemID","date"],inplace=True)

  return data

In [ ]:
'''def add_total_sales_and_promos(data, **kwargs):
  """
  Add function for total sales and total promotions per day
  """
  data2 = data.reset_index(drop=True)
  data2 = data.groupby(["date"]).agg({'order': 'sum','promoted': 'sum'})
  data2.rename(columns = {'order':'total_sales','promoted':'count_promotions'})
  data = data.merge(data2, on='date',how='left')
  
  return data'''

'def add_total_sales_and_promos(data, **kwargs):\n  """\n  Add function for total sales and total promotions per day\n  """\n  data2 = data.reset_index(drop=True)\n  data2 = data.groupby(["date"]).agg({\'order\': \'sum\',\'promoted\': \'sum\'})\n  data2.rename(columns = {\'order\':\'total_sales\',\'promoted\':\'count_promotions\'})\n  data = data.merge(data2, on=\'date\',how=\'left\')\n  \n  return data'

In [ ]:
def add_total_sales_and_promos(data, **kwargs):
  """
  Add function for total sales and total promotions per day
  """
  data2 = data.reset_index()
  data2 = data2[['date', 'order', 'promoted']]
  data2 = data2.groupby(["date"]).sum()
  data2 = data2.rename(columns = {'order':'total_sales','promoted':'count_promotions'})
  
  data = data.reset_index()
  data = data.join(data2, on='date',how='left')
  data = data.set_index(["itemID","date"])
  return data

In [ ]:
add_total_sales_and_promos(dynamic_feature)

order  salesPrice  promoted  total_sales  count_promotions
itemID date                                                                  
1      2018-01-01    0.0        3.11     False       6305.0              36.0
       2018-01-02    0.0        3.11     False       6165.0              46.0
       2018-01-03    0.0        3.11     False       2821.0              24.0
       2018-01-04    0.0        3.11     False      10819.0              54.0
       2018-01-05    0.0        3.11     False      11465.0              41.0
...                  ...         ...       ...          ...               ...
10463  2018-06-25    NaN      282.16     False          0.0             110.0
       2018-06-26    NaN      282.16     False          0.0             172.0
       2018-06-27    NaN      282.16     False          0.0             140.0
       2018-06-28    NaN      282.16     False          0.0             119.0
       2018-06-29    NaN      282.16     False          0.0             162.0

[1883340 rows x 5 columns]

In [ ]:
def forecast_total_sales(data, **kwargs):
  """
  forecast total sales for simulation period using prophet
  """
  #Prepare format for prophet
  data2 = data.reset_index(drop=True)
  data2 = data.groupby(["date"]).agg({'order': 'sum','promoted': 'sum'})
  data2.reset_index(inplace=True)
  data2.rename(columns = {'date':'ds','order':'y','promoted':'count_promotions'}, inplace = True) 

  #Prepare train and test data
  train = data2[:-14]
  test = data2[-14:]

  #Make Prediction
  from fbprophet import Prophet
  prophet_basic = Prophet()
  prophet_basic.add_regressor('count_promotions')
  prophet_basic.fit(train)
  forecast = prophet_basic.predict(test)

  #Add Forecast to original data
  data = data.reset_index()
  forecast['y'] = forecast['yhat']
  final_data = train.append(forecast[['ds','y','count_promotions']])
  final_data.rename(columns= {'ds':'date','y':'total_sales'}, inplace = True)
  data = data.merge(final_data, on='date',how='left')
  data.set_index(['itemID','date'], inplace = True)
  #data.drop(columns = ['level_0','index'], inplace = True)
  return data

# 7. Apply Features

Now here we can add the function which should be applied.
The function in the list, is executed in exactly this order. In case of a keyError it might be that one function is dependend on another function which is not yet executed. 

## Apply Function on Dynamic Features

Here the function which should be applied on the 'dynamic features' dataframe are specified.

In [ ]:
applied_features_for_dynamic_features = [
                    add_divided_time_feature,
                    add_two_days_after_promo, 
                    add_total_sales_and_promos,
                    ## forecast_total_sales, #only for final submission
                    # add_avg_daily_salesPrice, # add directly before applying dynamic features list, due to dependency with some static features
                    ### add_sold_per_period, # this fct doesn't exist? # Here order will be filled with nan -> Not good we need orders for test set to be Nan. -> This fct doesn't exists?
                    add_german_holidays,
                    # add_salesPrice_over_price, # add directly before applying dynamic features list, due to dependency with some static features
                    last_promotions,
                    add_price_bins(10, field='salesPrice')
]

## Apply Function on Items Table



In [ ]:
applied_features_for_items = [
                    add_price_sumOrder_cluster,
                    add_price_maxOrder_cluster,
                    # add_avg_salesPrice, # add directly before applying dynamic features, see next code cell
                    add_has_rating_feature,
                    add_price_bins(10,field='recommendedRetailPrice'),
                    ## add_avg_daily_sold,
                    add_bought_together_clusters,
                    # add_avg_of_daily_salesPrice, # add directly before applying dynamic features, see next code cell
                    num_promotions, 
                    promotion_in_data,
                    avg_sold_promoted,
                    ### avg_sold_not_promoted # columns overlap but no suffix specified: Index(['avg_sold_not_promoted'], dtype='object') in final_submission
]

## Technical Applying the features
This is the code to really use the above features.

In [ ]:
# dependent features, need to run in this order.. 
# apply first before calling the lists 

items = add_avg_salesPrice(items, orders)


... adding avg sales price


In [ ]:
dynamic_feature = add_avg_daily_salesPrice(dynamic_feature, orders, items)

... adding daily sales price


In [ ]:
items = add_avg_of_daily_salesPrice(items, dynamic_feature)
dynamic_feature = add_salesPrice_over_price(dynamic_feature, items)

... applying add_salesPrice_over_price


In [ ]:
def apply_features_training(data:dict, features: List[Callable])->dict:
  assert isinstance(features, list)

  backup_data = data.copy()
  
  try:
    for feature in features:
      result = feature(data, ors=orders, its=items, td=dynamic_feature)
      assert result is not None, "The function need to return the mutated data"
      data = result
    return data
  except Exception as e:
    print(f"\nERROR:\nIn Function {feature} was following Error:\n\n {e} \n Now reverting the data")
    return backup_data

print("+++ ADDING DYNAMIC FEATURES+++ \n _____________________________________________")
dynamic_feature = apply_features_training(dynamic_feature, applied_features_for_dynamic_features)
print("+++ ADDING STATIC FEATURES+++ \n _____________________________________________")
items = apply_features_training(items, applied_features_for_items)

assert dynamic_feature['order'].isnull().values.any()

+++ ADDING DYNAMIC FEATURES+++ 
 _____________________________________________
... adding time features
... applying add_german_holidays
... adding last_promotions
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
... applying price bins on salesPrice with 10 bins
+++ ADDING STATIC FEATURES+++ 
 _____________________________________________
... applying has rating
... applying price bins on recommendedRetailPrice with 10 bins
... adding bought_together_cluster
... adding num promotions
... adding promotion in data
                   order  salesPrice  ...  last_promoted_sales  salesPrice_bucket
itemID date                           ...                                        
1      2018-01-01    0.0        3.11  ...                164.0                  0
       2018-01-02    0.0        3.11  ...                164.0                  0
       2018-01-03    0.0        3.11  ...                164.0                  0
       2018-01-04    0.0        3.11  ...                164.0       

In [ ]:
dynamic_feature

order  salesPrice  ...  last_promoted_sales  salesPrice_bucket
itemID date                           ...                                        
1      2018-01-01    0.0        3.11  ...                164.0                  0
       2018-01-02    0.0        3.11  ...                164.0                  0
       2018-01-03    0.0        3.11  ...                164.0                  0
       2018-01-04    0.0        3.11  ...                164.0                  0
       2018-01-05    0.0        3.11  ...                164.0                  0
...                  ...         ...  ...                  ...                ...
10463  2018-06-25    NaN      282.16  ...                146.0                  9
       2018-06-26    NaN      282.16  ...                146.0                  9
       2018-06-27    NaN      282.16  ...                146.0                  9
       2018-06-28    NaN      282.16  ...                146.0                  9
       2018-06-29    NaN      282.16  ...                146.0                  9

[1883340 rows x 22 columns]

In [ ]:
#print("+++ ADDING STATIC FEATURES+++ \n _____________________________________________")
#items = apply_features_training(items, applied_features_for_items)

In [ ]:
#Here all values from 1 to 9 should exist
items.price_maxSales_cluster.value_counts()

In [ ]:
# Here I will still apply the automated features (using featuretools) 
# once all new features are created correctly in the static/items table (Min)
# this will build upon the features we created in items as well as the original orders table 
# the newly created features will be added to "static_features_final.pk" 

# Save dataset
Here the both datasets are saved.

The following variable is used to save the data

In [ ]:
# save_folder = "./data"

Following both datasets are saved. Please don't save other data from here. We are only using the two files as described above.

In [ ]:
# def save_data(df: pd.DataFrame, filename: str):
#   """
#   Saves data to the pickle file.
#   Sample call:
#   save_data(items, "items.pk")
#   """
#   filename = f"{save_folder}/{filename}"
#   df.to_pickle(filename)


# save_data(items, 'static_features_final.pk' if make_final_submission else 'static_features.pk')
# save_data(dynamic_feature, 'dynamic_features_final.pk' if make_final_submission else 'dynamic_features.pk')

# FINAL DATA
These are the finalized dataframes

## Static Features

In [ ]:
items

,brand,manufacturer,customerRating,category1,category2,category3,recommendedRetailPrice,avg_salesPrice_nonunique,avg_salesPrice,price_sumSales_cluster,price_maxSales_cluster,has_rating,recommendedRetailPrice_bucket,bought_together_cluster,num_promotions,promotion_in_data,avg_sold_promoted
itemID,,,,,,,,,,,,,,,,,
1,0,1,4.38,1,1,1,8.84,3.110000,3.110000,3,2,True,0,-1,4.0,True,164.500000
2,0,2,3.00,1,2,1,16.92,9.150000,9.150000,4,1,True,3,-1,0.0,False,135.892729
3,0,3,5.00,1,3,1,15.89,11.918371,12.001915,2,1,True,2,2,3.0,True,63.639457
4,0,2,4.44,1,2,1,40.17,13.010000,13.010000,5,4,True,6,-1,2.0,True,27.505000
5,0,2,2.33,1,1,1,17.04,7.740377,7.734465,5,2,True,3,-1,2.0,True,81.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10431,180,128,5.00,8,44,8,214.59,252.265000,252.188251,8,9,True,8,-1,3.0,True,228.843333
10459,180,253,0.00,8,44,8,56.57,14.710000,14.710000,7,7,False,7,-1,0.0,False,135.892729
10460,0,253,0.00,8,44,8,163.81,325.670000,325.670000,7,7,False,8,-1,0.0,False,135.892729


## Dynamic Features

In [ ]:
dynamic_feature

order  salesPrice  ...  last_promoted_sales  salesPrice_bucket
itemID date                            ...                                        
1      2018-01-01    0.00        3.11  ...                164.0                  0
       2018-01-02    0.00        3.11  ...                164.0                  0
       2018-01-03    0.00        3.11  ...                164.0                  0
       2018-01-04    0.00        3.11  ...                164.0                  0
       2018-01-05    0.00        3.11  ...                164.0                  0
...                   ...         ...  ...                  ...                ...
10463  2018-06-25  282.16      282.16  ...                130.0                  9
       2018-06-26  282.16      282.16  ...                130.0                  9
       2018-06-27  282.16      282.16  ...                130.0                  9
       2018-06-28  282.16      282.16  ...                130.0                  9
       2018-06-29  282.16      282.16  ...                130.0                  9

[1883340 rows x 22 columns]

In [ ]:
items.price_maxSales_cluster.value_counts()

7    2433
4    1468
3    1259
2     796
1     744
5     735
6     588
8     219
9     143
Name: price_maxSales_cluster, dtype: int64

In [ ]:
# !pip install featuretools

In [ ]:
import featuretools as ft

In [ ]:
entity_set = ft.EntitySet(id = 'itemID')

# create itemID as a column for featuretools 
items['itemID'] = items.index 

# create entity "items"
entity_set = entity_set.entity_from_dataframe(entity_id = 'items', 
                                              dataframe = items, 
                                              variable_types = {'brand': ft.variable_types.Categorical,
                                                                'manufacturer': ft.variable_types.Categorical,
                                                                'category1': ft.variable_types.Categorical,
                                                                'category2': ft.variable_types.Categorical,
                                                                'category3': ft.variable_types.Categorical},
                                              index = 'itemID') 

# create entity from the orders, with index "orderID" as no unique index exists
entity_set = entity_set.entity_from_dataframe(
                              entity_id = 'orders', 
                              dataframe = orders,
                              make_index = True,  # when unique index does not exist
                              index = 'orderID')
                              # time_index = 'time')

# Relationship: items - parent; orders - children
r_items_orders = ft.Relationship(entity_set['items']['itemID'], entity_set['orders']['itemID'])

# Add the relationship to the entity set
entity_set = entity_set.add_relationship(r_items_orders)

In [ ]:
# apply dfs or Deep Feature Synthesis: using basic building blocks (feature primitives) -> form new features. 
# The depth of a "deep feature" is equal to the number of stacked primitives.

static_features, static_features_names = ft.dfs(entityset=entity_set, target_entity='items', max_depth = 2)

# keep relevant features 
cols = [c for c in static_features.columns if ("transactID" not in c) and ("YEAR" not in c)]

static_features = static_features[cols]

In [ ]:
static_features

,customerRating,recommendedRetailPrice,avg_salesPrice_nonunique,avg_salesPrice,price_sumSales_cluster,price_maxSales_cluster,has_rating,bought_together_cluster,num_promotions,promotion_in_data,avg_sold_promoted,brand,manufacturer,category1,category2,category3,recommendedRetailPrice_bucket,SUM(orders.order),SUM(orders.salesPrice),STD(orders.order),STD(orders.salesPrice),MAX(orders.order),MAX(orders.salesPrice),SKEW(orders.order),SKEW(orders.salesPrice),MIN(orders.order),MIN(orders.salesPrice),MEAN(orders.order),MEAN(orders.salesPrice),COUNT(orders),NUM_UNIQUE(orders.DAY(time)),NUM_UNIQUE(orders.DAY(date)),NUM_UNIQUE(orders.MONTH(time)),NUM_UNIQUE(orders.MONTH(date)),NUM_UNIQUE(orders.WEEKDAY(time)),NUM_UNIQUE(orders.WEEKDAY(date)),MODE(orders.DAY(time)),MODE(orders.DAY(date)),MODE(orders.MONTH(time)),MODE(orders.MONTH(date)),MODE(orders.WEEKDAY(time)),MODE(orders.WEEKDAY(date))
itemID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.38,8.84,3.110000,3.110000,3,2,True,-1,4.0,True,164.500000,0,1,1,1,1,0,690,1788.25,0.483166,0.000000,4,3.11,2.603882,0.000000,1,3.11,1.200000,3.110000,575,20,20,5,5,7,7,29,29,1,1,0,0
2,3.00,16.92,9.150000,9.150000,4,1,True,-1,0.0,False,135.892729,0,2,1,2,1,3,5,45.75,0.000000,0.000000,1,9.15,0.000000,0.000000,1,9.15,1.000000,9.150000,5,4,4,2,2,4,4,29,29,5,5,1,1
3,5.00,15.89,11.918371,12.001915,2,1,True,2,3.0,True,63.639457,0,3,1,3,1,2,190,2121.47,0.251448,2.080328,2,14.04,3.479846,0.045338,1,9.89,1.067416,11.918371,178,11,11,4,4,6,6,18,18,1,1,3,3
4,4.44,40.17,13.010000,13.010000,5,4,True,-1,2.0,True,27.505000,0,2,1,2,1,6,53,650.50,0.239898,0.000000,2,13.01,3.821069,0.000000,1,13.01,1.060000,13.010000,50,8,8,4,4,4,4,22,22,4,4,6,6
5,2.33,17.04,7.740377,7.734465,5,2,True,-1,2.0,True,81.500000,0,2,1,1,1,3,177,1230.72,0.337172,0.161566,3,7.84,2.954571,-1.007652,1,7.48,1.113208,7.740377,159,9,9,4,4,7,7,17,17,4,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10431,5.00,214.59,252.265000,252.188251,8,9,True,-1,3.0,True,228.843333,180,128,8,44,8,8,189,46416.76,0.303568,12.904401,5,272.70,12.564062,0.953656,1,243.92,1.027174,252.265000,184,6,6,1,1,3,3,14,14,5,5,0,0
10459,0.00,56.57,14.710000,14.710000,7,7,False,-1,0.0,False,135.892729,180,253,8,44,8,7,1,14.71,NaN,NaN,1,14.71,NaN,NaN,1,14.71,1.000000,14.710000,1,1,1,1,1,1,1,5,5,5,5,5,5
10460,0.00,163.81,325.670000,325.670000,7,7,False,-1,0.0,False,135.892729,0,253,8,44,8,8,1,325.67,NaN,NaN,1,325.67,NaN,NaN,1,325.67,1.000000,325.670000,1,1,1,1,1,1,1,16,16,4,4,0,0


In [15]:
dynamic_testing = pd.read_pickle('./data/dynamic_features_final_nb2_2906_one_hot_ws7.pk')

In [16]:
dynamic_testing.reset_index(inplace = True)

In [62]:
dynamic_testing

,itemID,date,order,salesPrice,promoted,avg_daily_salesPrice,avg_daily_salesPrice_nonunique,salesPrice_over_recommended,salesPrice_over_avg_salesPrice,salesPrice_over_avg_salesPrice_nonunique,day_of_year,day_of_month,month,week_nr,promotion_lag1,promotion_lag2,two_days_after_promo,count_promotions_x,total_sales_y,de_holidays,days_since_last_promotion,last_promoted_sales,salesPrice_bucket__1,salesPrice_bucket__2,salesPrice_bucket__3,salesPrice_bucket__4,salesPrice_bucket__5,salesPrice_bucket__6,salesPrice_bucket__7,salesPrice_bucket__8,salesPrice_bucket__9,day_of_week__1,day_of_week__2,day_of_week__3,day_of_week__4,day_of_week__5,day_of_week__6,order_t-1,order_t-2,order_t-3,order_t-4,order_t-5,order_t-6,order_t-7
0,1,2018-01-01,0.0,8.84,0,3.111661,3.111661,0.351998,0.9985,1.0,1,1,1,1,0,0,0,36.0,6305.000000,0,31.0,164.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2018-01-02,0.0,8.84,0,3.111661,3.111661,0.351998,0.9985,1.0,2,2,1,1,0,0,0,46.0,6165.000000,0,32.0,164.0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2018-01-03,0.0,8.84,0,3.111661,3.111661,0.351998,0.9985,1.0,3,3,1,1,0,0,0,24.0,2821.000000,0,33.0,164.0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,1,2018-01-04,0.0,8.84,0,3.111661,3.111661,0.351998,0.9985,1.0,4,4,1,1,0,0,0,54.0,10819.000000,0,34.0,164.0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4,1,2018-01-05,0.0,8.84,0,3.111661,3.111661,0.351998,0.9985,1.0,5,5,1,1,0,0,0,41.0,11465.000000,0,35.0,164.0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2029817,10463,2018-07-09,NaN,282.16,0,282.160000,282.160000,1.822504,1.0000,1.0,190,9,7,28,0,0,0,114.0,17985.895627,0,220.0,36.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2029818,10463,2018-07-10,NaN,282.16,0,282.160000,282.160000,1.822504,1.0000,1.0,191,10,7,28,0,0,0,164.0,22061.844149,0,221.0,36.0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2029819,10463,2018-07-11,NaN,282.16,0,282.160000,282.160000,1.822504,1.0000,1.0,192,11,7,28,0,0,0,109.0,17433.409683,0,222.0,36.0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2029820,10463,2018-07-12,NaN,282.16,0,282.160000,282.160000,1.822504,1.0000,1.0,193,12,7,28,0,0,0,97.0,15124.293980,0,223.0,36.0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
promotions = dynamic_testing[(dynamic_testing['date'] > "2018-06-29") & (dynamic_testing['date'] < "2018-07-14")]

In [20]:
promotions[promotions["promoted"] == 1]

,itemID,date,order,salesPrice,promoted,avg_daily_salesPrice,avg_daily_salesPrice_nonunique,salesPrice_over_recommended,salesPrice_over_avg_salesPrice,salesPrice_over_avg_salesPrice_nonunique,day_of_year,day_of_month,month,week_nr,promotion_lag1,promotion_lag2,two_days_after_promo,count_promotions_x,total_sales_y,de_holidays,days_since_last_promotion,last_promoted_sales,salesPrice_bucket__1,salesPrice_bucket__2,salesPrice_bucket__3,salesPrice_bucket__4,salesPrice_bucket__5,salesPrice_bucket__6,salesPrice_bucket__7,salesPrice_bucket__8,salesPrice_bucket__9,day_of_week__1,day_of_week__2,day_of_week__3,day_of_week__4,day_of_week__5,day_of_week__6,order_t-1,order_t-2,order_t-3,order_t-4,order_t-5,order_t-6,order_t-7
1883703,26,2018-07-13,NaN,14.27,1,14.043837,14.043837,0.594574,0.998945,1.0,194,13,7,28,0,0,0,110.0,13433.552017,0,24.0,119.0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1883717,27,2018-07-13,NaN,1.64,1,1.638711,1.638711,0.301234,1.001048,1.0,194,13,7,28,0,0,0,110.0,13433.552017,0,137.0,40.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1883745,29,2018-07-13,NaN,56.13,1,56.130000,56.130000,1.922919,1.000000,1.0,194,13,7,28,0,0,0,110.0,13433.552017,0,116.0,24.0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1883817,35,2018-07-01,NaN,337.84,1,362.527988,362.527988,2.085172,1.007145,1.0,182,1,7,26,0,0,0,118.0,20866.076344,0,37.0,523.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1883818,35,2018-07-02,NaN,337.84,1,362.527988,362.527988,2.085172,1.007145,1.0,183,2,7,27,1,0,1,192.0,27601.578161,0,1.0,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2029594,10447,2018-07-10,NaN,44.99,1,257.880000,257.880000,1.000000,1.000000,1.0,191,10,7,28,0,0,0,164.0,22061.844149,0,221.0,16.0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2029608,10448,2018-07-10,NaN,63.00,1,275.080000,275.080000,1.000000,1.000000,1.0,191,10,7,28,0,0,0,164.0,22061.844149,0,221.0,16.0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2029636,10450,2018-07-10,NaN,44.43,1,55.187793,55.187793,1.500484,1.004895,1.0,191,10,7,28,0,0,0,164.0,22061.844149,0,25.0,121.0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2029707,10455,2018-07-11,NaN,62.84,1,1037.500000,1037.500000,1.000000,1.000000,1.0,192,11,7,28,0,0,0,109.0,17433.409683,0,222.0,16.0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
promotions[promotions["itemID"] == 81]

,itemID,date,order,salesPrice,promoted,avg_daily_salesPrice,avg_daily_salesPrice_nonunique,salesPrice_over_recommended,salesPrice_over_avg_salesPrice,salesPrice_over_avg_salesPrice_nonunique,day_of_year,day_of_month,month,week_nr,promotion_lag1,promotion_lag2,two_days_after_promo,count_promotions_x,total_sales_y,de_holidays,days_since_last_promotion,last_promoted_sales,salesPrice_bucket__1,salesPrice_bucket__2,salesPrice_bucket__3,salesPrice_bucket__4,salesPrice_bucket__5,salesPrice_bucket__6,salesPrice_bucket__7,salesPrice_bucket__8,salesPrice_bucket__9,day_of_week__1,day_of_week__2,day_of_week__3,day_of_week__4,day_of_week__5,day_of_week__6,order_t-1,order_t-2,order_t-3,order_t-4,order_t-5,order_t-6,order_t-7
1884460,81,2018-06-30,NaN,102.21,0,102.21,102.21,2.55015,1.0,1.0,181,30,6,26,0,0,0,123.0,18599.176207,0,11.0,155.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1884461,81,2018-07-01,NaN,102.21,0,102.21,102.21,2.55015,1.0,1.0,182,1,7,26,0,0,0,118.0,20866.076344,0,12.0,155.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1884462,81,2018-07-02,NaN,102.21,0,102.21,102.21,2.55015,1.0,1.0,183,2,7,27,0,0,0,192.0,27601.578161,0,13.0,155.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0
1884463,81,2018-07-03,NaN,102.21,0,102.21,102.21,2.55015,1.0,1.0,184,3,7,27,0,0,0,147.0,19953.405982,0,14.0,155.0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,NaN,NaN,NaN,0.0,0.0,0.0,0.0
1884464,81,2018-07-04,NaN,102.21,0,102.21,102.21,2.55015,1.0,1.0,185,4,7,27,0,0,0,240.0,33589.917451,0,15.0,155.0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1884465,81,2018-07-05,NaN,102.21,0,102.21,102.21,2.55015,1.0,1.0,186,5,7,27,0,0,0,179.0,25233.623701,0,16.0,155.0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1884466,81,2018-07-06,NaN,102.21,0,102.21,102.21,2.55015,1.0,1.0,187,6,7,27,0,0,0,179.0,21938.528379,0,17.0,155.0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1884467,81,2018-07-07,NaN,102.21,0,102.21,102.21,2.55015,1.0,1.0,188,7,7,27,0,0,0,141.0,20831.026171,0,18.0,155.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1884468,81,2018-07-08,NaN,102.21,0,102.21,102.21,2.55015,1.0,1.0,189,8,7,27,0,0,0,153.0,25195.926854,0,19.0,155.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1884469,81,2018-07-09,NaN,102.21,0,102.21,102.21,2.55015,1.0,1.0,190,9,7,28,0,0,0,114.0,17985.895627,0,20.0,155.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
info_promotions = info[~info['promotion'].isnull()]

info_promotions

,simulationPrice,promotion
itemID,,
26,14.27,2018-07-13
27,1.64,2018-07-13
29,56.13,2018-07-13
35,337.84,"2018-07-01,2018-07-02,2018-07-13"
58,277.72,2018-07-13
...,...,...
10447,44.99,2018-07-10
10448,63.00,2018-07-10
10450,44.43,2018-07-10


In [46]:
list_date = []

for index, row in info_promotions.iterrows():
    list_date.append(row["promotion"].split(","))
    # print(row["promotion"].split(","))

In [48]:
flat_date_list = [item for sublist in list_date for item in sublist]

In [50]:
len(flat_date_list)

2066

In [35]:
info[info.index == 81]

,simulationPrice,promotion
itemID,,
81,102.21,NaN


In [61]:
promotions[promotions["itemID"] == 276]

,itemID,date,order,salesPrice,promoted,avg_daily_salesPrice,avg_daily_salesPrice_nonunique,salesPrice_over_recommended,salesPrice_over_avg_salesPrice,salesPrice_over_avg_salesPrice_nonunique,day_of_year,day_of_month,month,week_nr,promotion_lag1,promotion_lag2,two_days_after_promo,count_promotions_x,total_sales_y,de_holidays,days_since_last_promotion,last_promoted_sales,salesPrice_bucket__1,salesPrice_bucket__2,salesPrice_bucket__3,salesPrice_bucket__4,salesPrice_bucket__5,salesPrice_bucket__6,salesPrice_bucket__7,salesPrice_bucket__8,salesPrice_bucket__9,day_of_week__1,day_of_week__2,day_of_week__3,day_of_week__4,day_of_week__5,day_of_week__6,order_t-1,order_t-2,order_t-3,order_t-4,order_t-5,order_t-6,order_t-7
1887190,276,2018-06-30,NaN,8813.07,0,8813.07,8813.07,1.418057,1.0,1.0,181,30,6,26,0,0,0,123.0,18599.176207,0,211.0,16.0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1887191,276,2018-07-01,NaN,8813.07,0,8813.07,8813.07,1.418057,1.0,1.0,182,1,7,26,0,0,0,118.0,20866.076344,0,212.0,16.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1887192,276,2018-07-02,NaN,8813.07,0,8813.07,8813.07,1.418057,1.0,1.0,183,2,7,27,0,0,0,192.0,27601.578161,0,213.0,16.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0
1887193,276,2018-07-03,NaN,8813.07,0,8813.07,8813.07,1.418057,1.0,1.0,184,3,7,27,0,0,0,147.0,19953.405982,0,214.0,16.0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,NaN,NaN,NaN,0.0,0.0,0.0,0.0
1887194,276,2018-07-04,NaN,8813.07,0,8813.07,8813.07,1.418057,1.0,1.0,185,4,7,27,0,0,0,240.0,33589.917451,0,215.0,16.0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1887195,276,2018-07-05,NaN,8813.07,0,8813.07,8813.07,1.418057,1.0,1.0,186,5,7,27,0,0,0,179.0,25233.623701,0,216.0,16.0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1887196,276,2018-07-06,NaN,8813.07,0,8813.07,8813.07,1.418057,1.0,1.0,187,6,7,27,0,0,0,179.0,21938.528379,0,217.0,16.0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1887197,276,2018-07-07,NaN,8813.07,0,8813.07,8813.07,1.418057,1.0,1.0,188,7,7,27,0,0,0,141.0,20831.026171,0,218.0,16.0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1887198,276,2018-07-08,NaN,8813.07,0,8813.07,8813.07,1.418057,1.0,1.0,189,8,7,27,0,0,0,153.0,25195.926854,0,219.0,16.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1887199,276,2018-07-09,NaN,8813.07,0,8813.07,8813.07,1.418057,1.0,1.0,190,9,7,28,0,0,0,114.0,17985.895627,0,220.0,16.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
info_promotions[info_promotions.index == 10060]

,simulationPrice,promotion
itemID,,


In [56]:
info[info.index == 81]

,simulationPrice,promotion
itemID,,
81,102.21,NaN


In [57]:
static_testing = pd.read_pickle('./data/static_features_final_nb2_2906_one_hot_ws7.pk')

In [59]:
static_testing[static_testing.index == 81]

,customerRating,recommendedRetailPrice,avg_salesPrice_nonunique,avg_salesPrice,avg_sold_promoted,has_rating,num_promotions,promotion_in_data,avg_sold_not_promoted,SUM(orders.order),SUM(orders.salesPrice),STD(orders.order),STD(orders.salesPrice),MAX(orders.order),MAX(orders.salesPrice),SKEW(orders.order),SKEW(orders.salesPrice),MIN(orders.order),MIN(orders.salesPrice),MEAN(orders.order),MEAN(orders.salesPrice),COUNT(orders),NUM_UNIQUE(orders.DAY(time)),NUM_UNIQUE(orders.DAY(date)),NUM_UNIQUE(orders.MONTH(time)),NUM_UNIQUE(orders.MONTH(date)),NUM_UNIQUE(orders.WEEKDAY(time)),NUM_UNIQUE(orders.WEEKDAY(date)),MODE(orders.DAY(time)),MODE(orders.DAY(date)),MODE(orders.MONTH(time)),MODE(orders.MONTH(date)),MODE(orders.WEEKDAY(time)),MODE(orders.WEEKDAY(date)),category1__2,category1__3,category1__4,category1__5,category1__6,category1__7,...,brand__274,recommendedRetailPrice_bucket__1,recommendedRetailPrice_bucket__2,recommendedRetailPrice_bucket__3,recommendedRetailPrice_bucket__4,recommendedRetailPrice_bucket__5,recommendedRetailPrice_bucket__6,recommendedRetailPrice_bucket__7,recommendedRetailPrice_bucket__8,recommendedRetailPrice_bucket__9,bought_together_cluster__0,bought_together_cluster__2,bought_together_cluster__5,bought_together_cluster__6,bought_together_cluster__8,bought_together_cluster__12,bought_together_cluster__13,bought_together_cluster__16,bought_together_cluster__21,bought_together_cluster__24,bought_together_cluster__26,bought_together_cluster__27,bought_together_cluster__28,bought_together_cluster__31,bought_together_cluster__32,bought_together_cluster__33,bought_together_cluster__34,bought_together_cluster__37,bought_together_cluster__38,bought_together_cluster__39,bought_together_cluster__40,bought_together_cluster__43,bought_together_cluster__46,bought_together_cluster__47,bought_together_cluster__49,bought_together_cluster__50,bought_together_cluster__51,bought_together_cluster__52,bought_together_cluster__53,bought_together_cluster__55
itemID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
81,5.0,40.08,102.21,102.21,126.0,1,3.0,1,0.033898,384.0,35773.5,0.315307,0.0,3.0,102.21,3.26974,0.0,1.0,102.21,1.097143,102.21,350.0,7.0,7.0,5.0,5.0,4.0,4.0,13.0,13.0,2.0,2.0,1.0,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [72]:
info[info.index == 359]

,simulationPrice,promotion
itemID,,
359,33.53,2018-06-30
